In [1]:
# others
import pandas as pd
import numpy as np
import argparse
import datetime
from copy import deepcopy # Add Deepcopy for args
import pickle 
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error

# pytorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim


print(torch.__version__)
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (15, 9)


# read file
df = pd.read_hdf('./data/merged_data.h5')

1.8.1
Populating the interactive namespace from numpy and matplotlib


In [2]:
''' greedy feature handleing'''
# test_df = train_x_df[train_x_df['volume'] != 0]
# test_df['rest_asset'] = test_df['volume'] - test_df['tb_base_av']
# test_df['greedy'] = test_df['tb_base_av'] / test_df['volume']

# test_df2 = test_df[['time', 'coin_index', 'open', 'high', 'low', 'close', 'volume', 'trades', 'tb_base_av','rest_asset', 'greedy']]
# test_df2[['coin_index','trades', 'volume', 'tb_base_av','rest_asset', 'greedy']].head()
# test_df2[test_df2['greedy'] == 1][['coin_index','trades', 'volume', 'tb_base_av','rest_asset', 'greedy']].head()

' greedy feature handleing'

In [4]:
train_df = df[df.sample_id < 1000]
val_df = df[(df.sample_id >= 1000) & (df.sample_id < 1108)]
test_df = df[df.sample_id >= 1108]

# 개수 체크
print(
    f'''
    train set is {len(train_df) / 1500}
    val set is {len(val_df) / 1500}
    test set is {len(test_df)/1500}
    '''
)


    train set is 1000.0
    val set is 108.0
    test set is 100.0
    


In [5]:
# ====== Generating Dataset ====== #

def data_gernerate(dataframe, x_frames, y_frames, print_mode = False):

    ''' 설명 생략 '''

    # grouping
    grouped_df = dataframe.groupby('sample_id')
    
    # export unique sample ids
    unique_sample_id_list = grouped_df.sample_id.unique()

    # create new lists
    X, y = list(), list()

    ''' 샘플 하나 선택 loop '''
    for sample_id in unique_sample_id_list:
        
        # get one sample_id in sample list
        temp_sample_id = sample_id.item()

        # get one group by temp_sample_id
        temp_df = grouped_df.get_group(temp_sample_id)

        # 한 샘플당 몇 개의 arrset가 나오는 지 확인
        count = 0
        split_length = len(temp_df) - (x_frames + y_frames) + 1
        
        ''' 한 샘플 내 데이터 split loop '''
        for time_idx in range(split_length):
            
            # index 변경
            time_idx += x_frames
            
            # temp_data select
            temp_arr = temp_df.iloc[time_idx - x_frames : time_idx + y_frames, 3:].values

            # get values
            temp_x = temp_arr[:x_frames, :]
            temp_y = temp_arr[x_frames:, :]

#             # 2d to 3d -> (255, 12) to (1, 255, 12) / (120, 12) to (1, 120, 12)
#             temp_3d_x = np.expand_dims(temp_2d_x, axis = 0)
#             temp_3d_y = np.expand_dims(temp_2d_y, axis = 0)
            
            # appending
            X.append(temp_x)
            y.append(temp_y)
            
            # counter printing
            count += 1
            if (count == split_length) & (print_mode == True):
                print(f'현재 sample id : {temp_sample_id}')
                print(f'{temp_sample_id}번째 sample의 생성 array수 : {count}')
            
    return np.array(X), np.array(y)

In [17]:
print(
    f'''
    {train_x_arr.shape}
    {train_y_arr[:,:,0].shape}
    '''
)



    (112600, 255, 9)
    (112600, 120)
    


In [37]:
class DatasetGenerater(Dataset):
    ''' 설명 생략 '''
    
    def __init__(self, dataframe, x_frames, y_frames):

        self.grouped_dataframe = dataframe.groupby('sample_id')
        self.sample_length = dataframe.sample_id.nunique()
        self.x_frames = x_frames
        self.y_frames = y_frames
        
        unique_sample_id_list = self.grouped_dataframe.sample_id.unique()
        
    def __len__(self):
        return self.sample_length
    
    def __getitem__(self, idx):
        
        one_sample_df = self.grouped_dataframe.get_group(idx)

        one_sample_arr = one_sample_df[-(self.x_frames + self.y_frames):].values

        # get values
        X = one_sample_arr[:self.x_frames, :]
        y = one_sample_arr[self.x_frames:, :]
        
        return X, y

In [42]:
trainset = DatasetGenerater(train_df, args.x_frames, args.y_frames)
trainloader = DataLoader(trainset, batch_size = args.bat, shuffle = True, drop_last = True)


In [43]:
for idx, (X, y)  in enumerate(trainloader):
    print(idx, X.shape, y.shape)

0 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
1 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
2 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
3 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
4 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
5 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
6 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
7 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
8 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
9 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
10 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
11 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
12 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
13 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
14 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
15 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
16 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
17 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
18 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
19 

165 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
166 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
167 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
168 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
169 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
170 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
171 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
172 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
173 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
174 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
175 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
176 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
177 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
178 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
179 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
180 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
181 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
182 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
183 torch.Size([1, 255, 12])

347 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
348 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
349 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
350 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
351 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
352 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
353 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
354 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
355 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
356 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
357 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
358 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
359 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
360 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
361 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
362 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
363 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
364 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
365 torch.Size([1, 255, 12])

545 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
546 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
547 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
548 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
549 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
550 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
551 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
552 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
553 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
554 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
555 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
556 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
557 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
558 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
559 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
560 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
561 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
562 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
563 torch.Size([1, 255, 12])

728 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
729 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
730 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
731 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
732 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
733 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
734 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
735 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
736 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
737 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
738 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
739 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
740 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
741 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
742 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
743 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
744 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
745 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
746 torch.Size([1, 255, 12])

941 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
942 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
943 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
944 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
945 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
946 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
947 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
948 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
949 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
950 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
951 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
952 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
953 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
954 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
955 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
956 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
957 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
958 torch.Size([1, 255, 12]) torch.Size([1, 120, 12])
959 torch.Size([1, 255, 12])

In [9]:
import encoder_decoder as modeler

# ====== initialization
parser = argparse.ArgumentParser()
args = parser.parse_args("")
args.device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("device is",args.device)

# ====== Model Capacity options ===== #
args.input_dim = 9
args.hidden_dim = 50
args.output_dim = 1
args.num_layers = 1
args.batch_size = 64
args.dropout = 0.2
args.target_len = 120
args.use_bn = True

# ====== Dataset Generating options ====== #
args.x_frames = 255
args.y_frames = 120

# ====== Model training options ===== #
args.num_epoch = 30
args.learning_rate = 0.001
args.L2_rate = 0.0001
args.training_prediction = 'mixed_teacher_forcing'
args.teacher_forcing_ratio = 0.6

device is cpu


In [11]:
# Encoder
enc = encoder_decoder.Encoder(args.input_dim, args.hidden_dim, args.num_layers)
# Decoder
dec = encoder_decoder.Decoder(args.output_dim, args.hidden_dim, args.num_layers, args.dropout, args.use_bn)

# Seq2Seq model
model = encoder_decoder.Seq2Seq(enc, dec, args.device, args.target_len, args.training_prediction, args.teacher_forcing_ratio)
model.to(args.device)


loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=args.learning_rate, weight_decay=args.L2_rate)
    

In [15]:
train_x_arr.shape

(112600, 255, 9)

In [14]:
for x, y in train_x_arr, train_y_arr:
    print(x.shape)
    break

ValueError: too many values to unpack (expected 2)

In [12]:

for i in range(args.num_epoch):
    model.train()
    model.zero_grad()
    optimizer.zero_grad()
    
    model.hidden = model.init_hidden()
    
    for x in train_x_arr:
        x = torch.Tensor(x).float()
        y_true = torch.Tensor(y_true).float()

        y_pred, hidden = model(x)
        model.hidden = hidden

    loss = loss_fn(y_pred.view(-1), y_true.view(-1))
    loss.backward()
    optimizer.step()
    print(loss.item())
    
    
        for i, (X, y) in enumerate(trainloader):
        
        X = X.transpose(0, 1).float().to(args.device)
        y_true = y[:,:,0].transpose(0, 1).float().to(args.device)

        # zero the gradient
        optimizer.zero_grad()
        
        # en-decoder outputs tensor 
        y_pred = model(X, y_true)

        # compute the loss 
        loss = loss_fn(y_true, y_pred)
        
        #backpropagation
        loss.backward()
        optimizer.step()
        
        # get the batch loss
        train_loss += loss.item()

AttributeError: 'Seq2Seq' object has no attribute 'init_hidden'

In [ ]:
# numpy arrary conversion

def df2d_to_array3d(df_2d):
    # 입력 받은 2차원 데이터 프레임을 3차원 numpy array로 변경하는 함수
    feature_size = df_2d.iloc[:,2:].shape[1]
    time_size = len(df_2d.time.value_counts())
    sample_size = len(df_2d.sample_id.value_counts())

    # 2위 팀에서 임의로 넣어둠
    # sample_index = df_2d.sample_id.value_counts().index
    array_3d = df_2d.iloc[:,2:].values.reshape([sample_size, time_size, feature_size])
    return array_3d


In [ ]:
print(
    f'''
    {df.high.max()}
    {df.low.max()}
    {df.open.max()}
    {df.close.max()}
    
    
    {df.high.min()}
    {df.low.min()}
    {df.open.min()}
    {df.close.min()}
    
    '''
    
    ''' high - low = 변동폭 \n'''
    ''' 음봉양봉 구분 추가 가능'''
)
